In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Activity')

+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
|     1     |     2     | 2016-03-01 |      5       |
|     1     |     2     | 2016-03-02 |      6       |
|     2     |     3     | 2017-06-25 |      1       |
|     3     |     1     | 2016-03-02 |      0       |
|     3     |     4     | 2018-07-03 |      5       |
+-----------+-----------+------------+--------------+


### Write an SQL query to report the fraction of players that logged in again `on the day after the day they first logged in`, rounded to 2 decimal places. 

#### *In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.*

```
+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+
Explanation: 
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33
```

In [3]:
%%sql 

SELECT 
    COUNT(DISTINCT player_id) AS player_count
FROM Activity


player_count
3


In [4]:
%%sql 

SELECT 
    event_date,
    MIN(event_date) OVER(PARTITION BY player_id) AS earliest_event_date
FROM Activity

event_date,earliest_event_date
2016-03-01,2016-03-01
2016-03-02,2016-03-01
2017-06-25,2017-06-25
2016-03-02,2016-03-02
2018-07-03,2016-03-02


In [5]:
%%sql 

SELECT 
    DATEDIFF(event_date, MIN(event_date) OVER (PARTITION BY player_id)) AS difference
FROM Activity;

difference
0
1
0
0
853


In [6]:
%%sql 

SELECT 
    RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS ranking 
FROM Activity;

ranking
1
2
1
1
2


In [7]:
%%sql 

SELECT 
    event_date,
    player_id,
    DATEDIFF(event_date, MIN(event_date) OVER(PARTITION BY player_id)) AS difference,
    RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS ranking
FROM Activity;

event_date,player_id,difference,ranking
2016-03-01,1,0,1
2016-03-02,1,1,2
2017-06-25,2,0,1
2016-03-02,3,0,1
2018-07-03,3,853,2


In [8]:
%%sql 

WITH cte_table AS(
    SELECT
        event_date,
        player_id,
        DATEDIFF(event_date, MIN(event_date) OVER (PARTITION BY player_id)) AS difference,
        RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS ranking
    FROM Activity
) SELECT 
    cte_table.difference
FROM Activity
JOIN cte_table ON cte_table.event_date = Activity.event_date AND cte_table.player_id = Activity.player_id 

difference
0
1
0
0
853


In [9]:
%%sql 

WITH cte_table AS(
    SELECT
        event_date,
        player_id,
        DATEDIFF(event_date, MIN(event_date) OVER (PARTITION BY player_id)) AS difference,
        RANK() OVER(PARTITION BY player_id ORDER BY event_date) AS ranking
    FROM Activity
) SELECT 
    Activity.event_date, 
    Activity.player_id, 
    Activity.event_date
FROM Activity
JOIN cte_table ON cte_table.event_date = Activity.event_date AND cte_table.player_id = Activity.player_id 
WHERE ranking=2 and difference = 1

event_date,player_id,event_date_1
2016-03-02,1,2016-03-02


In [10]:
%%sql 

WITH cte_table AS(
    SELECT
        event_date,
        player_id,
        DATEDIFF(event_date, MIN(event_date) OVER (PARTITION BY player_id)) AS difference,
        RANK() OVER(PARTITION BY player_id ORDER BY event_date) AS ranking
    FROM Activity
) SELECT 
    COUNT(cte_table.difference) / (SELECT COUNT(DISTINCT player_id) FROM Activity) AS fraction
FROM Activity
JOIN cte_table ON cte_table.event_date = Activity.event_date AND cte_table.player_id = Activity.player_id 
WHERE difference = 1

fraction
0.3333


In [11]:
%%sql 

WITH cte_table AS(
    SELECT
        event_date,
        player_id,
        DATEDIFF(event_date, MIN(event_date) OVER (PARTITION BY player_id)) AS difference,
        RANK() OVER(PARTITION BY player_id ORDER BY event_date) AS ranking
    FROM Activity
) SELECT 
    ROUND((COUNT(cte_table.difference) / (SELECT COUNT(DISTINCT player_id) FROM Activity)), 2) AS fraction
FROM Activity
JOIN cte_table ON cte_table.event_date = Activity.event_date AND cte_table.player_id = Activity.player_id 
WHERE difference = 1 AND ranking = 2

fraction
0.33


# Using Pandas

In [12]:
import pandas as pd 

In [13]:
activity_query = %sql SELECT * FROM Activity # type: ignore 
activity_df = activity_query.DataFrame()
display(activity_df)

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [14]:
activity_df.groupby('player_id').agg({'event_date': 'min'})

,event_date
player_id,
1,2016-03-01
2,2017-06-25
3,2016-03-02


In [15]:
grouped_df = activity_df \
            .groupby('player_id') \
            .agg({'event_date': 'min'}) \
            .reset_index()
grouped_df

,player_id,event_date
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


In [16]:
# Merge activity_df with grouped_df to get the minimum event_date for each player
merged_df = activity_df.merge(grouped_df, on='player_id', suffixes=['', '_min'])
merged_df

,player_id,device_id,event_date,games_played,event_date_min
0,1,2,2016-03-01,5,2016-03-01
1,1,2,2016-03-02,6,2016-03-01
2,2,3,2017-06-25,1,2017-06-25
3,3,1,2016-03-02,0,2016-03-02
4,3,4,2018-07-03,5,2016-03-02


In [17]:
# Calculate the date difference between event_date and the minimum event_date for each player
merged_df['date_diff'] = merged_df['event_date'] - merged_df['event_date_min']

merged_df

,player_id,device_id,event_date,games_played,event_date_min,date_diff
0,1,2,2016-03-01,5,2016-03-01,0:00:00
1,1,2,2016-03-02,6,2016-03-01,"1 day, 0:00:00"
2,2,3,2017-06-25,1,2017-06-25,0:00:00
3,3,1,2016-03-02,0,2016-03-02,0:00:00
4,3,4,2018-07-03,5,2016-03-02,"853 days, 0:00:00"


In [18]:
# Filter out the records where the date difference is equal to 1
filtered_df = merged_df[merged_df['date_diff'] == pd.Timedelta(days=1)]
filtered_df

,player_id,device_id,event_date,games_played,event_date_min,date_diff
1,1,2,2016-03-02,6,2016-03-01,"1 day, 0:00:00"


In [19]:
grouped_df.shape

(3, 2)

In [20]:
filtered_df.shape[0] / grouped_df.shape[0] 

0.3333333333333333

In [21]:
round(filtered_df.shape[0] / grouped_df.shape[0], 2)

0.33

## Using `mask`

In [22]:
# Calculate the minimum event_date for each player
min_event_date = activity_df.groupby('player_id')['event_date'].transform('min')
min_event_date

0    2016-03-01
1    2016-03-01
2    2017-06-25
3    2016-03-02
4    2016-03-02
Name: event_date, dtype: object

In [23]:
# Calculate the difference between event_date and the minimum event_date for each player
date_diff = activity_df['event_date'] - min_event_date
date_diff

0              0:00:00
1       1 day, 0:00:00
2              0:00:00
3              0:00:00
4    853 days, 0:00:00
Name: event_date, dtype: object

In [24]:
# Create a boolean mask based on the condition where the date_diff is equal to 1
mask = date_diff == pd.Timedelta(days=1)
mask

0    False
1     True
2    False
3    False
4    False
Name: event_date, dtype: bool

In [25]:
# Apply the boolean mask to filter the records
filtered_df = activity_df[mask]
filtered_df

,player_id,device_id,event_date,games_played
1,1,2,2016-03-02,6


In [26]:
# Filter out the records where the date difference is equal to 1
filtered_df = merged_df[merged_df['date_diff'] == pd.Timedelta(days=1)]
filtered_df

,player_id,device_id,event_date,games_played,event_date_min,date_diff
1,1,2,2016-03-02,6,2016-03-01,"1 day, 0:00:00"


In [27]:
filtered_df['player_id'].nunique()

1

In [28]:
fraction = round(filtered_df['player_id'].nunique() / activity_df['player_id'].nunique(), 2)
fraction


0.33